# General

In [ ]:
!pip install docx
!pip install python-docx
!pip install spacy
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 609.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=e6299cbf8c0a4d817285e0f57e8873fa93c6efcc33e38f4db3fbdc449820fd67
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 696.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=df91c12874502650d6aa53f615582b4304737ae739911bf18c092134ea737808
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import requests
import docx
import spacy
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

# Model 1 - Baseline - Bert Multilingual + Cosine search

In [ ]:
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load a pre-trained model tokenizer and model from Hugging Face transformers
model_name = "bert-base-multilingual-cased"  # Pre-trained multilingual BERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)

In [ ]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding

def vector_search(query_embedding, encoded_chunks, chunks):
    # Compute cosine similarity between query embedding and all chunk embeddings
    similarities = []

    for emb in encoded_chunks:
        sim = cosine_similarity(query_embedding.numpy().reshape(1, -1), emb.numpy().reshape(1, -1))
        similarities.append(sim.item())

    # Find indices of the top 3 most similar chunks
    closest_indices = np.argsort(similarities)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, closest_indices


# Model 2 - RuBert + Cosine search

In [ ]:
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load the Ruberta model and tokenizer from Hugging Face transformers
model_name = "cointegrated/rubert-tiny2"  # Pre-trained Ruberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)

In [ ]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding

def vector_search(query_embedding, encoded_chunks, chunks):
    # Compute cosine similarity between query embedding and all chunk embeddings
    similarities = []

    for emb in encoded_chunks:
        sim = cosine_similarity(query_embedding.numpy().reshape(1, -1), emb.numpy().reshape(1, -1))
        similarities.append(sim.item())

    # Find indices of the top 3 most similar chunks
    closest_indices = np.argsort(similarities)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, closest_indices

# Model 3 - RuBert + Euclidean search

In [ ]:
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load the Ruberta model and tokenizer from Hugging Face transformers
model_name = "cointegrated/rubert-tiny2"  # Pre-trained Ruberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)

In [ ]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding


import numpy as np
from sklearn.preprocessing import binarize


def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

def vector_search(query_embedding, encoded_chunks, chunks):
    distances = []

    for emb in encoded_chunks:
        dist = euclidean_distance(query_embedding, emb)
        distances.append(dist)

    # Find indices of the top 3 closest chunks (smallest distances)
    closest_indices = np.argsort(distances)[:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, closest_indices

# Model 4 - Jaccard search


In [ ]:
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunking(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks = chunking(document_text)

In [ ]:
import numpy as np

def jaccard_similarity(s1, s2):
    s1 = s1.lower().split(" ")
    s2 = s2.lower().split(" ")
    set1, set2 = set(s1), set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

def jacc_search(query, chunks):
    distances = []

    for chunk in chunks:
        dist = jaccard_similarity(query, chunk)
        distances.append(dist)

    # Find indices of the top 3 closest chunks (smallest distances)
    closest_indices = np.argsort(distances)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, distances,closest_indices

# query = 'что подрузумевается под прокторингом?'
# closest_chunks,dist,cl = jacc_search(query,chunks)

In [ ]:
def answer_4(query, chunks=chunks):

  closest_chunks,dist,cl = jacc_search(query,chunks)
  # print(f'Closest chunks: {cl}')

  context = str(closest_chunks)

  prompt = {
      "modelUri": "gpt://folder/yandexgpt-lite",
      "completionOptions": {
          "stream": False,
          "temperature": 0.6,
          "maxTokens": "2000"
      },
      "messages": [
          {
              "role": "system",
              "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи \
              вот ДОКУМЕНТЫ: " + context
          },
          {
              "role": "user",
              "text": query
          }
      ]
  }


  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key key"
  }

  response = requests.post(url, headers=headers, json=prompt)
  result = response.text
  response_data = response.json()
  assistant_text = response_data['result']['alternatives'][0]['message']['text']
  return assistant_text.replace('\n', '').replace('*', ''), cl

# General part

In [ ]:
i=1
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

chunk  1

1. 




 Положение об организации промежуточной аттестации и текущего контроля успеваемости студентов Национального исследовательского университета « Высшая школа экономики » 



























 Москва , 2023 

 Используемые понятия и сокращения 

 Положение – Положение об организации промежуточной аттестации и текущего контроля успеваемости студентов Национального исследовательского университета « Высшая школа экономики » . 
 НИУ ВШЭ , Университет – Национальный исследовательский университет « Высшая школа экономики » , в том числе филиалы . 
 Кампус – НИУ ВШЭ ( Москва ) , НИУ ВШЭ - Санкт - Петербург , НИУ ВШЭ - Нижний Новгород или НИУ ВШЭ - Пермь . 
 Образовательная программа – образовательная программа высшего образования – программа бакалавриата , специалитета , магистратуры . 
 Академический руководитель – работник НИУ ВШЭ из числа научно - педагогических работников , отвечающий за проектирование , реализацию , эффективность отдельной образовательной программы .

In [ ]:
def answer(query, encoded_chunks=encoded_chunks, chunks=chunks):
  query_embedding = encode_query(query)
  closest_chunks, cl = vector_search(query_embedding, encoded_chunks, chunks)

  # print(f'Closest indices: {cl}')

  context = str(closest_chunks)

  prompt = {
    "modelUri": "gpt://folder/yandexgpt-lite",
    "completionOptions": {
        "stream": False,
        "temperature": 0.6,
        "maxTokens": 2000
        },
    "messages": [
        {
            "role": "system",
            "text": """ты умный помошник студента, используй документы ниже,
            чтобы полноценно ответить на поставленный вопрос, если в документах этой информации нет,
            то просто скажи, что не знаешь ответа. Отвечай без вступления, либо сразу ответ на вопрос,
            либо сразу отвечаешь, что не знаешь ответа\
            вот ДОКУМЕНТЫ: """ + context
        },
        {
            "role": "user",
            "text": query
        }
        ]
    }

  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key key"
      }

  response = requests.post(url, headers=headers, json=prompt)
  result = response.text
  response_data = response.json()
  assistant_text = response_data['result']['alternatives'][0]['message']['text']
  # print(assistant_text)
  return assistant_text.replace('\n', '').replace('*', ''),  cl


In [ ]:
answer("Что такое сессия в учебном процессе НИУ ВШЭ?")

('Согласно приведённому вами тексту, сессия — это период, во время которого происходит сдача зачётов и экзаменов студентами.  В тексте упоминаются следующие виды сессий: промежуточная аттестация; текущий контроль успеваемости; осенний период пересдач; весенний период пересдач.Также в тексте подробно описан процесс организации и проведения промежуточной аттестации для студентов образовательной программы «Совместная программа по экономике НИУ ВШЭ и РЭШ» факультета экономических наук НИУ ВШЭ. Промежуточная аттестация включает в себя проведение независимых экзаменов по цифровой компетенции (НЭ по ЦК), которые состоят из трёх этапов:1. Входное тестирование.2. Промежуточное тестирование.3. Итоговое тестирование.Независимые экзамены оцениваются по десятибалльной шкале. Неудовлетворительная оценка влечёт за собой возникновение академической задолженности, которую необходимо устранить. Студенты, получившие неудовлетворительные оценки или не сдавшие экзамены, имеют право пересдать независимую оц

# Model 5 - Jaccard search + Lemmatization + YaGPT assistance

In [ ]:
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunking(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks = chunking(document_text)

In [ ]:
import numpy as np
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemma(text):
    lemmatized_words = []
    for word in text:
        parsed_word = morph.parse(word)[0]
        lemmatized_words.append(parsed_word.normal_form)
    return lemmatized_words

def jaccard_similarity(s1, s2):
    morph = pymorphy2.MorphAnalyzer()
    s1 = s1.lower().split(" ")
    s2 = s2.lower().split(" ")
    s1 = lemma(s1)
    s2= lemma(s2)
    set1, set2 = set(s1), set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

def jacc_search(query, chunks):
    distances = []

    for chunk in chunks:
        dist = jaccard_similarity(query, chunk)
        distances.append(dist)

    # Find indices of the top 3 closest chunks (smallest distances)
    closest_indices = np.argsort(distances)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, distances,closest_indices

In [ ]:
def prepare(query):

  prompt = {
      "modelUri": "gpt://folder/yandexgpt-lite",
      "completionOptions": {
          "stream": False,
          "temperature": 0.6,
          "maxTokens": "2000"
      },
      "messages": [
          {
              "role": "system",
              "text": "ты умный помошник студента, постарайся ответить на поставленный вопрос"
          },
          {
              "role": "user",
              "text": query
          }
      ]
  }


  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key key"
  }

  response = requests.post(url, headers=headers, json=prompt)
  result = response.text
  response_data = response.json()
  assistant_text = response_data['result']['alternatives'][0]['message']['text']
  return assistant_text.replace('\n', '').replace('*', '')

In [ ]:
def answer_5(query, chunks=chunks):
  closest_chunks,dist,cl = jacc_search(query+' '+ prepare(query),chunks)
  # print(f'Closest chunks: {cl}')

  context = str(closest_chunks)

  prompt = {
      "modelUri": "gpt://folder/yandexgpt-lite",
      "completionOptions": {
          "stream": False,
          "temperature": 0.6,
          "maxTokens": "2000"
      },
      "messages": [
          {
              "role": "system",
              "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи \
              вот ДОКУМЕНТЫ: " + context
          },
          {
              "role": "user",
              "text": query
          }
      ]
  }


  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key key"
  }

  response = requests.post(url, headers=headers, json=prompt)
  result = response.text
  response_data = response.json()
  assistant_text = response_data['result']['alternatives'][0]['message']['text']
  return assistant_text.replace('\n', '').replace('*', ''), cl

In [ ]:
query = "Что обозначает термин 'кампус' в контексте НИУ ВШЭ?"
answer_5(query, chunks)

('Термин «кампус» может относиться к университетскому городку, в котором расположены здания и инфраструктура для обучения и проживания студентов и сотрудников университета. Это может относится к территории, на которой находятся учебные корпуса Национального исследовательского университета «Высшая школа экономики». Однако в данном контексте термин не упоминается.Если у вас есть дополнительные вопросы, пожалуйста, уточните запрос.',
 array([ 8, 22, 25]))

# Model 6 - RuBert + Cosine search + QE

In [ ]:
# Load spaCy Russian tokenizer
nlp = spacy.blank("ru")

# Load the Ruberta model and tokenizer from Hugging Face transformers
model_name = "cointegrated/rubert-tiny2"  # Pre-trained Ruberta model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def read_docx(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Extract text content from the document
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Join all paragraphs into a single string
    document_text = "\n".join(full_text)
    return document_text

def chunk_and_encode_document(document_text, max_chunk_len=600):
    # Split the document into chunks
    doc = nlp(document_text)
    chunks = []
    current_chunk = []

    for token in doc:
        if len(current_chunk) + len(token.text) > max_chunk_len:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

        current_chunk.append(token.text)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Encode each chunk into vectors
    encoded_chunks = []

    for chunk in chunks:
        # Tokenize the chunk
        inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)

        # Pass the input through the model and get the hidden states
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (CLS token embedding)
        embeddings = outputs.last_hidden_state[:, 0, :]

        # Normalize the embeddings
        normalized_embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        # Append the normalized embedding to the list
        encoded_chunks.append(normalized_embeddings)

    return chunks, encoded_chunks

# Example usage
file_path = "main_doc.docx"
document_text = read_docx(file_path)
chunks, encoded_chunks = chunk_and_encode_document(document_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

In [ ]:
def encode_query(query_text):
    # Encode the query text into a vector
    inputs = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings (CLS token embedding)
    query_embedding = outputs.last_hidden_state[:, 0, :]

    # Normalize the embedding
    normalized_query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)

    return normalized_query_embedding



def vector_search(query_embedding, encoded_chunks, chunks):
    # Compute cosine similarity between query embedding and all chunk embeddings
    similarities = []

    for emb in encoded_chunks:
        sim = cosine_similarity(query_embedding.numpy().reshape(1, -1), emb.numpy().reshape(1, -1))
        similarities.append(sim.item())

    # Find indices of the top 3 most similar chunks
    closest_indices = np.argsort(similarities)[::-1][:3]
    closest_chunks = [chunks[idx] for idx in closest_indices]

    return closest_chunks, closest_indices


In [ ]:
def process(query):

  prompt = {
      "modelUri": "gpt://folder/yandexgpt-lite",
      "completionOptions": {
          "stream": False,
          "temperature": 0.6,
          "maxTokens": "2000"
      },
      "messages": [
          {
              "role": "system",
              "text": "ты умный помошник студента, постарайся ответить на поставленный вопрос"
          },
          {
              "role": "user",
              "text": query
          }
      ]
  }

  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key key"
  }

  response = requests.post(url, headers=headers, json=prompt)
  result = response.text
  response_data = response.json()
  assistant_text = response_data['result']['alternatives'][0]['message']['text'].replace('\n', '').replace('*', '')
  return assistant_text

In [ ]:
def answer_6(query, encoded_chunks=encoded_chunks, chunks=chunks):
  query = query + ' ' + process(query)
  query_embedding = encode_query(query)

  closest_chunks, closest_indices = vector_search(query_embedding, encoded_chunks, chunks)
  print(f"Closest chunks to the query: {closest_indices}")

  context = str(closest_chunks)

  prompt = {
      "modelUri": "gpt://folder/yandexgpt-lite",
      "completionOptions": {
          "stream": False,
          "temperature": 0.6,
          "maxTokens": "2000"
      },
      "messages": [
          {
              "role": "system",
              "text": "ты умный помошник студента, используй документы ниже, чтобы ответить на поставленный вопрос, если в документах этой информации нет, то так и скажи\
              вот ДОКУМЕНТЫ: " + context
          },
          {
              "role": "user",
              "text": query
          }
      ]
  }

  url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
  headers = {
      "Content-Type": "application/json",
      "Authorization": "Api-Key key"
  }

  response = requests.post(url, headers=headers, json=prompt)
  result = response.text
  response_data = response.json()
  assistant_text = response_data['result']['alternatives'][0]['message']['text'].replace('\n', '').replace('*', '')
  return assistant_text


In [ ]:

answer_6('Влияет ли положительная пересдача блокирующего элемента контроля на итоговую оценку студента?')

Closest chunks to the query: [4 5 3]


'На этот вопрос нет однозначного ответа, поскольку он зависит от конкретных правил и политики университета или образовательной программы. Из приведённого вами документа можно сделать вывод, что блокирующие элементы контроля — это те, которые могут заблокировать часть промежуточной оценки. Если блокирующий элемент контроля будет пересдан, то это может повлиять на итоговую оценку, поскольку она зависит от всех элементов контроля.Однако, из документа также следует, что оценку за элемент контроля, объявленную преподавателем студенту, не может изменить, кроме случаев, предусмотренных Положением. Поэтому, если блокирующий элемент контроля уже был объявлен, и студент его успешно пересдал, то это не повлияет на итоговую оценку.Рекомендуется обратиться к административным источникам, например, к преподавателю, руководителю департамента или администрации университета, для получения точной и актуальной информации о влиянии успешной пересдачи блокирующего элемента на итоговую оценку по дисциплине в

# Metrics


In [ ]:
import pandas as pd

# df = pd.read_excel('context_question_answer BASELINE Model 1.xlsx')
df = pd.read_excel('context_question_answer RuBert_Cosine Model 2.xlsx')
# df = pd.read_excel('context_question_answer RuBert_Euclidean Model 3.xlsx')
# df = pd.read_excel('context_question_answer Jaccard Model 4.xlsx')
# df = pd.read_excel('context_question_answer 1 Jaccard_Lemma_QE Model 5.xlsx')
# df = pd.read_excel('context_question_answer 2 Jaccard_Lemma_QE Model 5.xlsx')
# df = pd.read_excel('context_question_answer 3 Jaccard_Lemma_QE Model 5.xlsx')
# df = pd.read_excel('context_question_answer 4 Jaccard_Lemma_QE Model 5.xlsx')
# df = pd.read_excel('context_question_answer 5 Jaccard_Lemma_QE Model 5.xlsx')
# df = pd.read_excel('context_question_answer 1 RuBert_Cosine_QE Model 6.xlsx')
# df = pd.read_excel('context_question_answer 2 RuBert_Cosine_QE Model 6.xlsx')
# df = pd.read_excel('context_question_answer 3 RuBert_Cosine_QE Model 6.xlsx')
# df = pd.read_excel('context_question_answer 4 RuBert_Cosine_QE Model 6.xlsx')
# df = pd.read_excel('context_question_answer 5 RuBert_Cosine_QE Model 6.xlsx')

In [ ]:
df['Model response'] = [0]*61
df

,Context,ctop1,ctop2,ctop3,Query,Answer,top 1,top 2,top 3,rel 1,rel 2,rel 3,Model response
0,Определение 'Положение' относится к Положению ...,1,3,16,Что означает термин 'Положение' в контексте до...,Термин 'Положение' в контексте документа означ...,2,21,1,0,0,1,0
1,НИУ ВШЭ Университет' включает в себя Националь...,1,2,10,Что включает в себя понятие 'НИУ ВШЭ Университ...,Понятие 'НИУ ВШЭ Университет' включает в себя ...,1,9,21,1,0,0,0
2,Академический руководитель – это работник НИУ ...,1,3,13,Какова роль академического руководителя в НИУ ...,Академический руководитель в НИУ ВШЭ отвечает ...,9,2,29,0,0,0,0
3,Кампус – НИУ ВШЭ (Москва) НИУ ВШЭ-Санкт-Петерб...,1,3,10,Что представляют собой кампусы НИУ ВШЭ?,Кампусы НИУ ВШЭ представляют собой филиалы уни...,21,9,1,0,0,1,0
4,Декан факультета – руководитель факультета,1,12,10,Какую роль выполняет декан факультета в НИУ ВШЭ?,Декан факультета в НИУ ВШЭ является руководите...,29,21,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Академическая задолженность – неудовлетворител...,1,19,15,Влияет ли академическая задолженность на возмо...,"Да, наличие академической задолженности может ...",42,2,37,0,0,0,0
57,Блокирующий Элемент контроля – элемент контрол...,1,2,10,Что такое блокирующий элемент контроля в учебн...,Блокирующий элемент контроля в учебной дисципл...,9,1,3,0,1,0,0
58,Блокирующий Элемент контроля – элемент контрол...,2,5,7,Каковы последствия получения неудовлетворитель...,Получение неудовлетворительной оценки по блоки...,42,2,25,0,1,0,0
59,Блокирующий Элемент контроля – элемент контрол...,1,7,10,Может ли студент исправить оценку по блокирующ...,"Да, студент в НИУ ВШЭ может исправить оценку п...",13,8,29,0,0,0,0


## Distance Metrics

In [ ]:
query = 'Что подразумевают под пререквизитами в образовательном процессе НИУ ВШЭ?'
predicted_text = answer(query)
actual_text = 'Пререквизиты в образовательных программах НИУ ВШЭ подразумевают набор дисциплин и компетенций, которые студент должен освоить перед началом обучения по конкретной дисциплине. Эти дисциплины называются дисциплинами-пререквизитами и обеспечивают необходимую базу знаний для успешного усвоения новых материалов.'
predicted_text

'Пререквизиты — это перечень результатов обучения или компетенций, которыми должен обладать студент, а также список учебных дисциплин, которые студент должен пройти перед началом изучения определённой учебной дисциплины или элемента образовательной программы.Пререквизиты нужны для того, чтобы обеспечить логическую последовательность и преемственность в образовательном процессе, а также для того, чтобы студент получил необходимые знания и навыки для успешного изучения последующих дисциплин.Например, студент, который изучает дисциплину «Математика 1», должен иметь базовые знания и навыки по математике и уметь выполнять основные математические операции. Но для того, чтобы он смог изучить дисциплину «Статистика 1», ему необходимо предварительно освоить дисциплину «Информатика 1», которая содержит в себе основы работы с компьютером и программным обеспечением. Поэтому «Информатика 1» становится пререквизитом для «Статистики 1».Блокирующий элемент контроля — это элемент контроля, который явля

In [ ]:
predicted = []

In [ ]:
k = len(predicted)
k

36

In [ ]:
for i in range(k, len(df)):
    a, cl = answer(df['Query'][i])
    predicted.append(a)
    df.loc[i,'Model response'] = a
    df.loc[i, 'top 1'] = cl[0]
    df.loc[i, 'top 2'] = cl[1]
    df.loc[i, 'top 3'] = cl[2]
    print(i, ': ', a)

36 :  Я не знаю точного ответа на этот вопрос. Но я могу сказать, что из документов, которые вы предоставили, следует, что ИУП — это индивидуальный учебный план студента. Он включает в себя перечень учебных дисциплин, практик и иных видов учебной работы, формы контроля и отведённые на них сроки. ИУП составляется с учётом индивидуальных особенностей и образовательных потребностей студента.ИУП может включать в себя: учебные дисциплины, предусмотренные базовым учебным планом образовательных программ, утверждённых до 2021-2022 года; дисциплины, не предусмотренные базовым учебным планом, но необходимые для подготовки к сдаче независимых экзаменов по программированию и анализу данных; перенос независимых экзаменов по программированию и анализу данных, пропущенных студентом в связи с академическим отпуском, отпуском по беременности и родам, отпуску по уходу за ребёнком; независимые экзамены по программированию и анализу данных без включения в индивидуальный учебный план студента дисциплин, на

In [ ]:
# df[12:]

In [ ]:
# predicted = [answer(df['Query'][i]) for i in range(len(df))]
# df['Model response'] = predicted
answer = [df['Answer'][i] for i in range(len(df))]

In [ ]:
df.to_excel('context_question_answer RuBert_Cosine Model 2.xlsx', index=False)

### Cosine similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def cosine_sim(predicted_text, actual_text):
  vectorizer = TfidfVectorizer()
  tfidf = vectorizer.fit_transform([predicted_text, actual_text])
  sim = cosine_similarity(tfidf[0:1], tfidf[1:2])
  return sim

def cosine(predicted, answer):
  return (sum(cosine_sim(predicted[i], answer[i]) for i in range(len(predicted)))/len(predicted))[0][0]

# print("Косинусное сходство:", cosine(predicted, answer))

In [ ]:
cosine_sim(predicted_text, actual_text)

array([[0.20233695]])

### Euclidian distance

In [ ]:
predicted = [predicted_text]
answer = [actual_text]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def euclidean_distance(v1, v2):
    return np.sqrt(np.sum((np.array(v1) - np.array(v2))**2))

def euclidean(predicted, answer):
  sum_distance = 0
  for i in range(len(predicted)):
    vectorizer = TfidfVectorizer()
    tfidf_vectors = vectorizer.fit_transform([predicted[i], answer[i]])
    distance = euclidean_distances(tfidf_vectors[0], tfidf_vectors[1])
    sum_distance += distance
  return (sum_distance/len(predicted))[0][0]

In [ ]:
print("Euclidian distance:", euclidean(predicted, answer))

Euclidian distance: 1.2630621918672678


### Jaccard similarity

In [ ]:
def jaccard_similarity(s1, s2):
    s1 = s1.lower().split(" ")
    s2 = s2.lower().split(" ")
    set1, set2 = set(s1), set(s2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

def jaccard(predicted, answer):
  return sum(jaccard_similarity(predicted[i], answer[i]) for i in range(len(predicted)))/len(predicted)

print("Jaccard similarity:", jaccard(predicted, answer))

Jaccard similarity: 0.09826589595375723


## Ranking Metrics

In [ ]:
query_embedding = encode_query(query)
query = 'Влияет ли положительная пересдача блокирующего элемента контроля на итоговую оценку студента?'
query = query.replace('\n', '').replace('*', '')
closest_chunks = vector_search(encode_query(query), encoded_chunks, chunks)
i=1
print("Closest chunks to the query:")
for idx, chunk in enumerate(closest_chunks, start=1):
    print('chunk ',i)
    print()
    i+=1
    print(f"{idx}. {chunk}")
    print()

Closest chunks to the query:
chunk  1

1. МАГОЛЕГО , общеуниверситетских факультативов , общефакультетских пулов ) , если эти Дисциплины реализуются одновременно для студентов разных образовательных программ . 
 При положительном согласовании изменения основных характеристик дисциплины Ответственный преподаватель обязан проинформировать студентов на странице дисциплины в ЭИОС не позднее , чем за один учебный день до начала реализации дисциплины с измененными характеристиками . 
 Характеристика системы оценивания и принципов расчета оценок 
 
 В Университете для выставления промежуточных и окончательных оценок по Дисциплине используется десятибалльная система . Правило округления может быть зафиксировано в ПУД . Если в ПУД отсутствует правило округления , то применятся арифметическое округление . 
 Десятибалльная система оценивания сопоставляется с качественной ( пятибалльной ) системой оценивания для целей указания в документах об обучении или об образовании , выдаваемых студентам по и

### AP@k

In [ ]:
def apk(actual, predicted, k=10):
    """
    Вычисляет среднюю точность на уровне k (AP@K) для одного запроса.

    :param actual: список релевантных элементов
    :param predicted: список предсказанных элементов
    :param k: количество топ элементов для учета в метрике
    :return: AP@K для данного запроса
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [ ]:
actual =[1, 2, 15]
generated = [1, 2, 38]

In [ ]:
actual = [df['ctop1'][1], df['ctop2'][1], df['ctop3'][1]]
generated = [df['top 1'][1], df['top 2'][1], df['top 3'][1]]

In [ ]:
apk(actual, generated, 3)

0.6666666666666666

### MAP@k

In [ ]:
actual = [[df['ctop1'][i], df['ctop2'][i], df['ctop3'][i]] for i in range(len(df))]
generated = [[df['top 1'][i], df['top 2'][i], df['top 3'][i]] for i in range(len(df))]

In [ ]:
def mapk(actual, predicted, k=10):
    """
    Вычисляет среднюю точность на уровне k (MAP@K) по всем запросам.

    :param actual: список списков релевантных элементов
    :param predicted: список списков предсказанных элементов
    :param k: количество топ элементов для учета в метрике
    :return: MAP@K по всем запросам
    """
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

k = 3

print("MAP@K:", mapk(actual, generated, k))

TypeError: object of type 'int' has no len()

### MRR

In [ ]:
def calculate_mrr(relevance):
    """
    Рассчитывает MRR для заданного списка релевантности.

    :param relevance: Список релевантности. Каждый элемент списка является списком логических значений,
                      где True указывает на релевантность результата для соответствующего запроса.
    :return: Значение MRR.
    """
    reciprocal_ranks = []

    for query_relevance in relevance:
        # Найдем первый релевантный результат
        for rank, is_relevant in enumerate(query_relevance, start=1):
            if is_relevant:
                reciprocal_ranks.append(1 / rank)
                break
        else:
            # Если нет релевантных результатов, добавим 0
            reciprocal_ranks.append(0)

    # Вычислим среднее значение обратных рангов
    mrr = sum(reciprocal_ranks) / len(relevance)
    return mrr

# Пример использования функции
# relevance = [[df['rel 1'][i], df['rel 2'][i], df['rel 3'][i]] for i in range(len(df))]

# mrr = calculate_mrr(relevance)
# print(f'MRR: {mrr}')

In [ ]:
relevance = [[1,1,0]]
calculate_mrr(relevance)

1.0

### DCG

In [ ]:
def dcg_at_k(relevance, k=3):
    """
    Рассчитывает DCG для заданного списка релевантности до k-го результата.

    :param relevance: Список релевантности.
    :param k: Количество результатов для расчета DCG.
    :return: Значение DCG.
    """
    relevance = np.asarray(relevance)[:k]
    dcg = np.sum(relevance / np.log2(np.arange(1, len(relevance) + 1) + 1))
    return dcg

In [ ]:
dcg_value = dcg_at_k(relevance)
print(f'DCG@{3}: {dcg_value}')

DCG@3: 2.0


### NDCG

In [ ]:
def ndcg_at_k(relevance, k=3):
    """
    Рассчитывает NDCG для заданного списка релевантности до k-го результата.

    :param relevance: Список релевантности.
    :param k: Количество результатов для расчета NDCG.
    :return: Значение NDCG.
    """
    dcg_max = dcg_at_k(sorted(relevance, reverse=True), k)
    if not dcg_max:
        return 0.0
    return dcg_at_k(relevance, k) / dcg_max

In [ ]:
ndcg_value = ndcg_at_k(relevance)
print(f'NDCG@{3}: {ndcg_value}')

NDCG@3: 1.0


## Metrics based on pre-trained LLMs (ROUGE, BERTScore)

In [ ]:
hypotheses = predicted
references = answer

### Rouge

In [ ]:
!pip install rouge
from rouge import Rouge

def evaluate_rouge(hypothesis, reference):
    rouge = Rouge()
    return rouge.get_scores(hypothesis, reference, avg=True)

# evaluate_rouge(hypotheses, references)

In [ ]:
evaluate_rouge(predicted_text, actual_text)

{'rouge-1': {'r': 0.2647058823529412,
  'p': 0.10843373493975904,
  'f': 0.15384614972313546},
 'rouge-2': {'r': 0.058823529411764705,
  'p': 0.02197802197802198,
  'f': 0.0319999960396805},
 'rouge-l': {'r': 0.23529411764705882,
  'p': 0.0963855421686747,
  'f': 0.1367521326291184}}

### BERTScore

In [ ]:
!pip install bert_score
from bert_score import score

def evaluate_bert_score(hypotheses, references, lang='ru'):
    P, R, F1 = score(hypotheses, references, lang=lang, rescale_with_baseline=True)
    return P.mean(), R.mean(), F1.mean()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
bert_scores = evaluate_bert_score(hypotheses, references, lang='ru')
print("Precision:", bert_scores[0])
print("Recall:", bert_scores[1])
print("F1 Score:", bert_scores[2])

Precision: tensor(0.6639)
Recall: tensor(0.7081)
F1 Score: tensor(0.6844)


In [ ]:
bert_scores = evaluate_bert_score([predicted_text], [actual_text], lang='ru')
print("Precision:", bert_scores[0])
print("Recall:", bert_scores[1])
print("F1 Score:", bert_scores[2])

Precision: tensor(0.6998)
Recall: tensor(0.7514)
F1 Score: tensor(0.7247)
